In [8]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
from tensorflow import keras
#import tensorflow_hub as hub
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator as IDG

import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2

In [2]:
WorkingDir = 'F:/Preprocessed/'
TestFrames1DIR = WorkingDir + 'TestFrames1.2/'
TestFrames2DIR = WorkingDir + 'TestFrames2.2/'
TestFrames3DIR = WorkingDir + 'TestFrames3.2/'
TrainFrames1DIR = WorkingDir + 'TrainFrames1.2/'
TrainFrames2DIR = WorkingDir + 'TestFrames2.2/'
TrainFrames3DIR = WorkingDir + 'TestFrames3.2/'

CATEGORIES = ['ApplyEyeMakeup', 'ApplyLipstick', 'Archery', 'BabyCrawling', 'BalanceBeam', 'BandMarching',
              'BaseballPitch', 'Basketball', 'BasketballDunk', 'BenchPress', 'Biking', 'Billiards', 'BlowDryHair',
              'BlowingCandles', 'BodyWeightSquats', 'Bowling', 'BoxingPunchingBag', 'BoxingSpeedBag', 'Breaststroke',
              'BrushingTeeth', 'CleanandJerk', 'CliffDiving', 'CricketBowling', 'CricketShot', 'CuttingInKitchen',
              'Diving', 'Drumming', 'Fencing', 'FieldHockeyPenalty', 'FloorGymnastics', 'FrisbeeCatch', 'FrontCrawl',
              'GolfSwing', 'Haircut', 'HammerThrow', 'Hammering', 'HandstandPushups', 'HandstandWalking',
              'HeadMassage', 'HighJump', 'HorseRace', 'HorseRiding', 'HulaHoop', 'IceDancing', 'JavelinThrow',
              'JugglingBalls', 'JumpRope', 'JumpingJack', 'Kayaking', 'Knitting', 'LongJump', 'Lunges',
              'MilitaryParade', 'Mixing', 'MoppingFloor', 'Nunchucks', 'ParallelBars', 'PizzaTossing',
              'PlayingGuitar', 'PlayingPiano', 'PlayingTabla', 'PlayingViolin', 'PlayingCello', 'PlayingDaf',
              'PlayingDhol', 'PlayingFlute', 'PlayingSitar', 'PoleVault', 'PommelHorse', 'PullUps', 'Punch',
              'PushUps', 'Rafting', 'RockClimbingIndoor', 'RopeClimbing', 'Rowing', 'SalsaSpin', 'ShavingBeard',
              'Shotput', 'SkateBoarding', 'Skiing', 'Skijet', 'SkyDiving', 'SoccerJuggling', 'SoccerPenalty',
              'StillRings', 'SumoWrestling', 'Surfing', 'Swing', 'TableTennisShot', 'TaiChi', 'TennisSwing',
              'ThrowDiscus', 'TrampolineJumping', 'Typing', 'UnevenBars', 'VolleyballSpiking', 'WalkingWithDog',
              'WallPushups', 'WritingOnBoard', 'YoYo']


In [3]:
def load_videos(directory, max_categories=101): # by default uses all categories
    data = []
    cat_num = 0
    
    for category in CATEGORIES:
        if(cat_num >= max_categories):
            break
        pathCat = os.path.join(directory, category)
        pathVid = directory + str(category)

        print( str(cat_num) + ': ' + category )
        for video in os.listdir(pathVid):
            pathImg = pathVid + '/' + str(video)
            
            for img in os.listdir(pathImg):
                img_array = cv2.imread(pathImg + '/' + img)
                data.append([np.array(img_array), cat_num])
            
        cat_num += 1
    print('done')
    return data

In [4]:
def sort_by_frame(testing_data):
    X = []
    y = []


    for features, label in testing_data:
        # put data from 0-255 into scale of 0-1
        X.append(features / 255.0)
        y.append(label)


    X = np.array(X) # convert to numpy array

    i=0
    # 20 arrays
    # X2[0] holds frame 0 of each video, X2[1] holds frame 1, etc.
    X2 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]

    # holds the category tag of the videos; y2[0] is for the 0th frames, etc.
    y2 = []

    # split each viedo along the columns of X2
    for frame in X:
        X2[ i % 20 ].append(frame)

        if i % 20 == 0:
            # add the video's tag to y2
            y2.append(y[i]) 

        i+=1
        if i % 1000 == 0:
            print('frame ' + str(i) + ' processed')

    X = None # free up storage

    print('done')
    return X2, y2

In [5]:
testing_data1 = load_videos(TestFrames1DIR)

0: ApplyEyeMakeup
1: ApplyLipstick
2: Archery
3: BabyCrawling
4: BalanceBeam
5: BandMarching
6: BaseballPitch
7: Basketball
8: BasketballDunk
9: BenchPress
10: Biking
11: Billiards
12: BlowDryHair
13: BlowingCandles
14: BodyWeightSquats
15: Bowling
16: BoxingPunchingBag
17: BoxingSpeedBag
18: Breaststroke
19: BrushingTeeth
20: CleanandJerk
21: CliffDiving
22: CricketBowling
23: CricketShot
24: CuttingInKitchen
25: Diving
26: Drumming
27: Fencing
28: FieldHockeyPenalty
29: FloorGymnastics
30: FrisbeeCatch
31: FrontCrawl
32: GolfSwing
33: Haircut
34: HammerThrow
35: Hammering
36: HandstandPushups
37: HandstandWalking
38: HeadMassage
39: HighJump
40: HorseRace
41: HorseRiding
42: HulaHoop
43: IceDancing
44: JavelinThrow
45: JugglingBalls
46: JumpRope
47: JumpingJack
48: Kayaking
49: Knitting
50: LongJump
51: Lunges
52: MilitaryParade
53: Mixing
54: MoppingFloor
55: Nunchucks
56: ParallelBars
57: PizzaTossing
58: PlayingGuitar
59: PlayingPiano
60: PlayingTabla
61: PlayingViolin
62: Playing

In [11]:
apple = ImageDataGenerator()
for image in testing_data1:
    apple.apply_transform(image, flip_horizontal)

NameError: name 'ImageDataGenerator' is not defined